# Exploratory Data Analisis (EDA)
En este notebook me enfoco en:
- Recolección de datos desde distintas fuentes
- Identificado diversas estructuras de los datasets
- Análisis de nulos, duplicados, relaciones
- Checklist de problemas identificados

## 1. Configuración inicial
- Importación de librerias

In [1]:
# Importado de librerías
import pandas as pd
import os
import json
import subprocess
import tabula

## 2. Obtención de datasets

- Obtención del dataset de ventas y paso a DataFrame

In [3]:
# Cargar credenciales de Kaggle desde un archivo JSON
with open("kaggle_keys.json") as f:
    kaggle_creds = json.load(f)

os.environ["KAGGLE_USERNAME"] = kaggle_creds["username"]
os.environ["KAGGLE_KEY"] = kaggle_creds["key"]

dataset = "virtualschool/restaurant-cost-and-sales-dataset"

# Ejecuta el comando kaggle datasets download
subprocess.run([
    "kaggle", "datasets", "download",
    "-d", dataset,
    "--unzip",
    "-p", 'data/raw'
])

dir_raw = r"data\raw"
excel_restaurante = dir_raw + r"\Restaurant_Data.xlsx"

df_orders = pd.read_excel(excel_restaurante, sheet_name="Orders")
df_items = pd.read_excel(excel_restaurante, sheet_name="Items")

- Dataset del INDEC

In [4]:
path_informe = dir_raw + r"\INFORME_INDEC_1-10-25.pdf"

# Obtener DataFrame que me interesa
df_indec = tabula.read_pdf(
    path_informe,
    pages=12,
    multiple_tables=True,
    encoding='latin1'  
)[0]

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

- Valor del dólar

In [ ]:
from utils import realizar_request

url_dolarapi = 'https://dolarapi.com/v1/dolares/blue'

request_ok, data = realizar_request(url_dolarapi)
if not request_ok:
    raise RuntimeError("No se pudo obtener el valor del dólar desde la API.")
dolar_blue = float(data.get("venta"))

## 3. Analizado de DataFrames y detección de tareas
- Vista general

In [ ]:
print(df_orders.info())
print('------------------')
print(df_items.info())
print('------------------')
print(df_indec.info())

NameError: name 'df_orders' is not defined

> Dado a lo que pareciera ser un problema en las columnas del INDEC, realizo una vista más detallada

In [ ]:
print(df_indec.head())

> Los nombres de las columnas parecieran no estar perfectamente alineados con el encabezado del DataFrame, pero están presentes!

- Ahora, **detección de nulos y duplicados**:

In [ ]:
print("Cantidad de filas y columnas - ", "Órdenes: ", df_orders.shape, "- Items: ", df_items.shape,"- INDEC: ", df_indec.shape)
print("-------------------")
print("Cantidad de valores nulos - ", "Órdenes: ", df_orders.isnull().sum().sum(), "- Items: ", df_items.isnull().sum().sum(),"- INDEC: ", df_indec.isnull().sum().sum())
print("-------------------")
print("Cantidad de duplicados - ", "Órdenes: ", df_orders.duplicated().sum(), "- Items: ", df_items.duplicated().sum(),"- INDEC: ", df_indec.duplicated().sum())

## 4. Pasado a formato CSV para la posterior realización de tareas

In [ ]:
staging_dir = r"data\staging"

# Almacenado de los DataFrames en formato CSV
df_orders.to_csv(os.path.join(staging_dir, "orders.csv"), index=False)
df_items.to_csv(os.path.join(staging_dir, "items.csv"), index=False)
df_indec.to_csv(os.path.join(staging_dir, "indec.csv"), index=False)

# Guardo el valor del dólar blue en un archivo de texto
with open(os.path.join(staging_dir, "dolar_blue.txt"), "w") as f:
    f.write(str(dolar_blue))


## 5. Síntesis del Exploratory Data Analysis

**Se puede apreciar fácilmente:**
- Diferencias de lenguaje: INDEC en español, pero ventas en inglés
- Diferencias entre formatos (preferible pasarlo todo a `snake_case`)
- Valores nulos varios: 77 en el INDEC y 6 en Items
- Formateo necesario del DataFrame del INDEC con asignación de columnas
- Relación entre Items (dimensión) y Orders (factos)

**A realizar:**
- Asignación de columnas que me interesan del INDEC (`Variedad`, `Unidad de medida` y `Octubre de 2025`)
- Limpieza de valores nulos
- Traducción al español del dataset de Kaggle
- Formateo a `snake_case` y eliminación de tildes
- Pasado de USD a ARS
- Generalización de unidades de medida a kilos.
